In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np
import warnings


warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

csv_file_path = "/slugbot_energy_data.csv"
df = pd.read_csv(csv_file_path)

label_encoders = {}
for col in ["Weather", "Energy Mode"]:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le


feature_columns = ["Hour", "Solar Energy (W)", "Biomass Energy (W)", "Battery Storage (W)", "Weather", "Day/Night", "Energy Mode"]
X = df[feature_columns].copy()
y = []


for _, row in df.iterrows():
    power_required = {0: 4, 1: 6, 2: 10}[row["Energy Mode"]]

    if row["Biomass Energy (W)"] >= power_required:
        y.append("Use Biomass Energy")
    elif row["Day/Night"] and row["Solar Energy (W)"] >= power_required:
        y.append("Use Solar Energy")
    elif row["Battery Storage (W)"] >= power_required:
        y.append("Use Battery Backup")
    else:
        y.append("Energy Deficit - Reduce Operations")

le_target = LabelEncoder()
y_encoded = le_target.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)


def predict_energy_decision(hour, weather, is_day, solar_energy, biomass_energy, battery_storage, mode):
    weather_encoded = label_encoders["Weather"].transform([weather])[0]
    mode_encoded = label_encoders["Energy Mode"].transform([mode])[0]
    input_data = pd.DataFrame([[hour, solar_energy, biomass_energy, battery_storage, weather_encoded, is_day, mode_encoded]], columns=feature_columns)
    prediction = model.predict(input_data)
    return le_target.inverse_transform(prediction)[0]

while True:
    print("\nEnergy Decision Prediction System")
    print("1. Predict using past dataset values")
    print("2. Enter Weather & Day/Night to predict")
    print("3. Enter Temperature separately to predict")
    print("4. Exit")
    choice = int(input("Enter your choice: "))

    if choice == 1:
        print("Using past dataset values for prediction...")
        print("Model trained successfully.")
    elif choice == 2:
        hour = int(input("Enter Hour (0-23): "))
        weather = input("Enter Weather (Sunny, Cloudy, Rainy): ")
        is_day = int(input("Enter Day/Night (1 for Day, 0 for Night): "))
        solar_energy = float(input("Enter Solar Energy (W): "))
        biomass_energy = float(input("Enter Biomass Energy (W): "))
        battery_storage = float(input("Enter Battery Storage (W): "))
        mode = input("Enter Energy Mode (Low Power, Standard Power, High Power): ")
        result = predict_energy_decision(hour, weather, is_day, solar_energy, biomass_energy, battery_storage, mode)
        print("Recommended Action:", result)
    elif choice == 3:
        try:
            temp = float(input("Enter Temperature (°C): "))

            if temp > 30:
                prediction = "Use Solar Energy (High Temperature)"
            elif 15 <= temp <= 30:
                prediction = "Use Mixed Mode (Solar + Biomass)"
            else:
                prediction = "Use Biomass Energy (Cold Weather)"

            print(f"\nPredicted Decision based on Temperature: {prediction}")

        except ValueError:
            print("Invalid input! Please enter a valid temperature.")
    elif choice == 4:
        print("Exiting system...")
        break
    else:
        print("Invalid choice. Please try again.")


Energy Decision Prediction System
1. Predict using past dataset values
2. Enter Weather & Day/Night to predict
3. Enter Temperature separately to predict
4. Exit
Enter your choice: 3
Enter Temperature (°C): 30

Predicted Decision based on Temperature: Use Mixed Mode (Solar + Biomass)

Energy Decision Prediction System
1. Predict using past dataset values
2. Enter Weather & Day/Night to predict
3. Enter Temperature separately to predict
4. Exit
Enter your choice: 1
Using past dataset values for prediction...
Model trained successfully.

Energy Decision Prediction System
1. Predict using past dataset values
2. Enter Weather & Day/Night to predict
3. Enter Temperature separately to predict
4. Exit
Enter your choice: 2
Enter Hour (0-23): 12
Enter Weather (Sunny, Cloudy, Rainy): Sunny
Enter Day/Night (1 for Day, 0 for Night): 1
Enter Solar Energy (W): 40
Enter Biomass Energy (W): 40
Enter Battery Storage (W): 50
Enter Energy Mode (Low Power, Standard Power, High Power): Low Power
Recommend